# Import packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import auc, roc_curve, RocCurveDisplay, r2_score,\
    mean_absolute_error, mean_squared_error, recall_score, accuracy_score, f1_score, \
    precision_score, balanced_accuracy_score, roc_auc_score

import seaborn as sns

# Custom functions

In [2]:
from pickle_managment import load_pickle
from model_evaluation import delongs_test

# Regression

## F-stat & p-values

# Classification

## DeLong's test

### Load models

In [3]:
centroid_svm = load_pickle(r'model_pickles\best_svm_classifier_centroid.pkl')

mlp_classifier = load_pickle(r'model_pickles\best_mlp_classifier.pkl')

X_classifier = load_pickle(r'model_outputs\svm_classifier\centroid_x_test.pkl')
y_classifier = load_pickle(r'model_outputs\svm_classifier\centroid_y_test.pkl')

### Prep holdout data

In [4]:
# classification_holdout_df = pd.read_csv(
#     r'datasets\holdout_datasets\classification_df_expanded_cleaned_holdout.csv.zip'
# )
# classification_holdout_df

In [5]:
# classification_holdout_X = classification_holdout_df.drop(['SMILES', 'BBB+/BBB-'],axis=1)
# 
# centroid_pipeline = load_pickle(
#     r'model_outputs\svm_classifier\centroid_pipeline.pkl'
# )
# X_processed = centroid_pipeline.transform(classification_holdout_X)
# 
# classification_holdout_y = classification_holdout_df['BBB+/BBB-']


In [6]:
centroid_probs = centroid_svm.predict_proba(X_classifier)

mlp_probs = mlp_classifier.predict_proba(X_classifier)

centroid_auc = roc_auc_score(y_classifier, centroid_probs[:, 1])

mlp_auc = roc_auc_score(y_classifier, mlp_probs[:, 1])

In [7]:
new_centroid_probs = centroid_probs[:,1]
new_mlp_probs = mlp_probs[:,1]

In [15]:
y_true = y_classifier.values
np.delete(y_true, 1)

for i in range(len(y_true)):
    if y_true[i] == 'BBB+':
        y_true[i] = 1
    else:
        y_true[i] = 0


In [16]:
delongs_test(new_centroid_probs, new_mlp_probs, centroid_auc, mlp_auc, y_classifier)

Model A's V10: [0.007782101167315175, 0.03696498054474708, 0.12840466926070038, 0.18093385214007782, 0.0, 0.10700389105058365, 0.011673151750972763, 0.0019455252918287938, 0.0038910505836575876, 0.0, 0.0038910505836575876, 0.01556420233463035, 0.0038910505836575876, 0.11089494163424124, 0.24124513618677043, 0.0, 0.03696498054474708, 0.017509727626459144, 0.04669260700389105, 0.007782101167315175, 0.0019455252918287938, 0.2607003891050584, 0.009727626459143969, 0.009727626459143969, 0.01556420233463035, 0.21595330739299612, 0.0038910505836575876, 0.048638132295719845, 0.017509727626459144, 0.0, 0.03696498054474708, 0.0019455252918287938, 0.009727626459143969, 0.011673151750972763, 0.0019455252918287938, 0.009727626459143969, 0.007782101167315175, 0.027237354085603113, 0.009727626459143969, 0.04669260700389105, 0.3638132295719844, 0.013618677042801557, 0.10311284046692606, 0.0, 0.0622568093385214, 0.03501945525291829, 0.017509727626459144, 0.017509727626459144, 0.03501945525291829, 0.110

0.0626839316363882

In [20]:
positive_label_index = 0
y_mlp_probabilities=mlp_classifier.predict_proba(X_classifier)[:, positive_label_index]
y_svm_probabilities=centroid_svm.predict_proba(X_classifier)[:, positive_label_index]

# Compute ROC curve and ROC area for each class
fpr_mlp, tpr_mlp, _ = roc_curve(y_classifier, y_mlp_probabilities,pos_label='BBB+')
fpr_svm, tpr_svm, _ = roc_curve(y_classifier, y_svm_probabilities,pos_label='BBB+')
roc_auc_mlp = auc(fpr_mlp, tpr_mlp)
roc_auc_svm = auc(fpr_svm, tpr_svm)

fig, ax = plt.subplots(figsize=(8, 8))

plt.plot(
    fpr_mlp,
    tpr_mlp,
    lw=4,
    label='MLP ROC curve (area = %0.2f)' % roc_auc_mlp
)
plt.plot(
    fpr_svm,
    tpr_svm,
    lw=4,
    label='SVM ROC curve (area = %0.2f)' % roc_auc_svm
)
plt.plot(
    [0, 1],
    [0, 1],
    color='green',
    lw=4,
    linestyle='--'
)

plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False positive rate',weight='bold', fontsize=12)
plt.ylabel('True positive rate',weight='bold', fontsize=12)
plt.title('MLP vs. SVM AUROC',weight='bold', fontsize=14)
plt.legend(loc='lower right')

# plt.savefig(
#     r'model_outputs\mlp_classifier\auroc.png',
#     dpi=600,
#     bbox_inches='tight'
# )

plt.show()

ValueError: unknown format is not supported